In [8]:
import pprint
import jieba

text_corpus = [
    "此前拜登政府宣布将投入几千亿美元推动美国电动汽车行业发展",
    "解放军东部战区新闻发言人表示台湾及其附属岛屿是中国领土的一部分",
    "受利好消息刺激影响海南概念板块直线上行券商股拉升",
    "要做好疫情防控毫不放松抓实抓细疫情防控各项措施",
    "上海市出台高中招考新政实验性示范性高中将出其招生计划",
    "游客出游热情高涨红色旅游持续升温踏青游近郊游乡村游自驾游等需求加速释放",
    "忙为新剧拍摄表示完成拍摄后返来电视城为劲歌金曲担任嘉宾亦很开心见到偶像",
    "印度菜对于香港人并不陌生咖喱薄饼串烧等印度美食更深得港人欢心",
    "继上一圈斗巴塞隆拿两回合狂轰四球安巴比这次面对盟主拜仁慕尼黑又有佳作",
]

stoplist = set('的 要 等 为'.split())

texts = [[word for word in jieba.cut(document, cut_all=False) if word not in stoplist] for document in text_corpus]
pprint.pprint(texts)

[['此前',
  '拜登',
  '政府',
  '宣布',
  '将',
  '投入',
  '几千',
  '亿美元',
  '推动',
  '美国',
  '电动汽车',
  '行业',
  '发展'],
 ['解放军',
  '东部',
  '战区',
  '新闻',
  '发言人',
  '表示',
  '台湾',
  '及其',
  '附属',
  '岛屿',
  '是',
  '中国',
  '领土',
  '一部分'],
 ['受', '利好', '消息', '刺激', '影响', '海南', '概念', '板块', '直线', '上行', '券商', '股', '拉升'],
 ['做好', '疫情', '防控', '毫不放松', '抓实', '抓细', '疫情', '防控', '各项措施'],
 ['上海市', '出台', '高中', '招考', '新政', '实验性', '示范性', '高中', '将', '出', '其', '招生', '计划'],
 ['游客',
  '出游',
  '热情高涨',
  '红色旅游',
  '持续',
  '升温',
  '踏青',
  '游',
  '近郊',
  '游',
  '乡村',
  '游',
  '自驾游',
  '需求',
  '加速',
  '释放'],
 ['忙',
  '新剧',
  '拍摄',
  '表示',
  '完成',
  '拍摄',
  '后',
  '返来',
  '电视',
  '城为',
  '劲歌',
  '金曲',
  '担任',
  '嘉宾',
  '亦',
  '很',
  '开心',
  '见到',
  '偶像'],
 ['印度',
  '菜',
  '对于',
  '香港',
  '人',
  '并',
  '不',
  '陌生',
  '咖喱',
  '薄饼',
  '串烧',
  '印度',
  '美食',
  '更深',
  '得',
  '港人',
  '欢心'],
 ['继上',
  '一圈',
  '斗',
  '巴塞隆',
  '拿',
  '两',
  '回合',
  '狂',
  '轰',
  '四球',
  '安',
  '巴比',
  '这次',
  '面对',
  '盟主',
  '拜仁',
  '慕尼黑',
  '又',
  '有',

In [9]:
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
pprint.pprint(processed_corpus)

[['将'],
 ['表示'],
 [],
 ['疫情', '防控', '疫情', '防控'],
 ['高中', '高中', '将'],
 ['游', '游', '游'],
 ['拍摄', '表示', '拍摄'],
 ['印度', '印度'],
 []]


In [10]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary(8 unique tokens: ['将', '表示', '疫情', '防控', '高中']...)


In [11]:
pprint.pprint(dictionary.token2id)

{'印度': 7, '将': 0, '拍摄': 6, '游': 5, '疫情': 2, '表示': 1, '防控': 3, '高中': 4}


In [12]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
pprint.pprint(bow_corpus)

[[(0, 1)],
 [(1, 1)],
 [],
 [(2, 2), (3, 2)],
 [(0, 1), (4, 2)],
 [(5, 3)],
 [(1, 1), (6, 2)],
 [(7, 2)],
 []]


In [13]:
from gensim import models

# train the model
tfidf = models.TfidfModel(bow_corpus)

# transform the "system minors" string
words = "system minors".lower().split()
print(tfidf[dictionary.doc2bow(words)])

[]


In [15]:
from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=8)

In [19]:
query_document = list(jieba.cut('拍摄印度', cut_all=False))
print(query_document)
query_bow = dictionary.doc2bow(query_document)
sims = index[tfidf[query_bow]]
print(list(enumerate(sims)))

['拍摄', '印度']
[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.6690057), (7, 0.70710677), (8, 0.0)]
